# Vetorização TF-IDF

**Objetivo:** Gerar vetorização TF-IDF da base pré-processada e realizar análise exploratória.


In [1]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.feature_extraction.text import TfidfVectorizer


## 1. Carregar dataset pré-processado


In [2]:
# Carregar dados pré-processados
with open('../data/processed/20news_preprocessed.pkl', 'rb') as f:
    data = pickle.load(f)

X_text = data['text']
y = data['target']
target_names = data['target_names']

print(f"Total de documentos: {len(X_text)}")
print(f"Classes: {target_names}")
print(f"Distribuição: {pd.Series(y).value_counts().sort_index().to_dict()}")


Total de documentos: 5611
Classes: ['rec.autos', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.space', 'talk.politics.guns', 'talk.politics.mideast']
Distribuição: {0: 931, 1: 951, 2: 973, 3: 954, 4: 885, 5: 917}


## 2. Criar TfidfVectorizer e gerar matriz TF-IDF


In [3]:
# Configurar TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),  # unigramas e bigramas
    min_df=5,  # palavra deve aparecer em pelo menos 5 documentos
    max_df=0.95,  # ignorar palavras que aparecem em mais de 95% dos documentos
    lowercase=True
)

# Gerar matriz TF-IDF
print("Gerando matriz TF-IDF...")
X_tfidf = vectorizer.fit_transform(X_text)

print(f"Shape da matriz TF-IDF: {X_tfidf.shape}")
print(f"Número de features: {len(vectorizer.get_feature_names_out())}")
print(f"Tipo: {type(X_tfidf)}")


Gerando matriz TF-IDF...
Shape da matriz TF-IDF: (5611, 5000)
Número de features: 5000
Tipo: <class 'scipy.sparse._csr.csr_matrix'>


## 3. Análise exploratória: Top palavras por classe


In [4]:
# Calcular média dos scores TF-IDF por classe
feature_names = vectorizer.get_feature_names_out()
top_words_per_class = {}

for class_idx, class_name in enumerate(target_names):
    # Índices dos documentos desta classe
    class_mask = y == class_idx
    # Média dos scores TF-IDF para esta classe
    class_tfidf = X_tfidf[class_mask].mean(axis=0).A1
    
    # Obter top 20 palavras
    top_indices = np.argsort(class_tfidf)[::-1][:20]
    top_words = [(feature_names[i], class_tfidf[i]) for i in top_indices]
    top_words_per_class[class_name] = top_words

# Exibir resultados
for class_name, words in top_words_per_class.items():
    print(f"\n{'='*60}")
    print(f"Top 20 palavras para: {class_name}")
    print(f"{'='*60}")
    for word, score in words:
        print(f"  {word:30s} {score:.4f}")



Top 20 palavras para: rec.autos
  car                            0.0716
  cars                           0.0319
  engine                         0.0260
  like                           0.0227
  just                           0.0221
  new                            0.0204
  know                           0.0193
  ford                           0.0182
  oil                            0.0177
  good                           0.0177
  dealer                         0.0176
  don                            0.0163
  think                          0.0156
  price                          0.0144
  does                           0.0142
  problem                        0.0137
  drive                          0.0135
  speed                          0.0133
  thanks                         0.0131
  right                          0.0118

Top 20 palavras para: rec.sport.baseball
  year                           0.0305
  game                           0.0297
  baseball                       0.0285
  gam

## 4. Salvar vetores TF-IDF


In [5]:
# Criar diretório se não existir
os.makedirs('../data/processed', exist_ok=True)

# Preparar dados para salvar
data_to_save = {
    'X_tfidf': X_tfidf,
    'y': y,
    'target_names': target_names,
    'feature_names': feature_names
}

# Salvar em pickle
output_path = '../data/processed/tfidf_vectors.pkl'
with open(output_path, 'wb') as f:
    pickle.dump(data_to_save, f)

print(f"Vetores TF-IDF salvos em: {output_path}")
print(f"Shape da matriz salva: {X_tfidf.shape}")


Vetores TF-IDF salvos em: ../data/processed/tfidf_vectors.pkl
Shape da matriz salva: (5611, 5000)
